<a href="https://colab.research.google.com/github/MatufA/bereshit-simulation/blob/master/bereshit_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bereshit space craft - Simulation
a basic flight controller of the Bereshit space craft.

In [0]:
from math import radians, sin, cos

In [0]:
class Moon:
  # from: https://he.wikipedia.org/wiki/%D7%94%D7%99%D7%A8%D7%97
  RADIUS = 3475*1000  # meters
  ACC = 1.622        # m/s^2
  EQ_SPEED = 1700    # m/s
  
  @staticmethod
  def get_acc(speed):
    num = abs(speed) / Moon.EQ_SPEED
    return (1 - num) * Moon.ACC

In [0]:
class Bereshit:
  WEIGHT_EMP = 165                        # kg.
  WEIGHT_FULE = 420                       # kg.
  WEIGHT_FULL = WEIGHT_EMP + WEIGHT_FULE  # kg.
  
  MAIN_ENG_F = 430                        # N.
  SECOND_ENG_F = 25                       # N.
  MAIN_BURN = 0.15                        # liter per sec, 12 liter per m.
  SECOND_BURN = 0.009                     # liter per sec 0.6 liter per m.
  ALL_BURN = MAIN_BURN + 8 * SECOND_BURN  # main engin and all seconds engins in work.
  
  @staticmethod
  def acc(weight, main, seconds):
    total = Bereshit.MAIN_ENG_F if main else 0
    total += seconds * Bereshit.SECOND_ENG_F
    return total / weight
  
  @staticmethod
  def acc_max(weight):
    return Bereshit.acc(weight, True, 8)
  
  def __init__(self, vs, hs, dist, ang, alt):
    self.vs = vs
    self.hs = hs
    self.dist = dist
    self.ang = ang    # zero is vertical (as in landing).
    self.alt = alt
    self.dt = 1       # sec.
    self.acc=0        # Acceleration rate (m/s^2).
    self.fuel = 121
    self.weight = Bereshit.WEIGHT_EMP + self.fuel
    self.NN = 0.7     # rate[0,1]
  
  def land(self):
    # over 2 km above the ground.
    if self.alt > 2000:
      # maintain a vertical speed of [20-25] m/s
      self.vs += 0.002 * self.dt if self.vs > 25 else 0
      self.vs -= 0.002 * self.dt if self.vs < 20 else 0

    else:
      # lower than 2 km - horizontal speed should be close to zero.
      self.ang -= 3 if self.ang > 3 else self.ang
      self.NN = 0.6  # brake slowly.
      self.hs = 0 if self.hs < 2 else self.hs
      
      if self.alt < 125:
        # very close to thr ground.
        self.NN = 0.7 if self.vs < 5 else 1 
        self.acc = 0 if self.vs < 5 else self.acc

    self.NN = .4 if self.alt < 5 else self.NN

    ang_rad = radians(self.ang) * self.acc
    h_acc = sin(ang_rad) * self.acc
    v_acc = cos(ang_rad) * self.acc
    moon_acc = Moon.get_acc(self.hs)

    dw = self.dt * Bereshit.ALL_BURN * self.NN
    if self.fuel:
      self.fuel -= dw
      self.weight = Bereshit.WEIGHT_EMP + self.fuel
      self.acc = self.NN * Bereshit.acc_max(self.weight)

    else:
      self.acc = 0

    v_acc -= moon_acc
    self.hs -= h_acc * self.dt if self.hs > 0 else 0
    self.dist -= self.hs * self.dt
    self.vs -= v_acc * self.dt
    self.alt -= self.dt * self.vs
      
  def __str__(self):
    return "{vs:.3f}, {hs:.3f}, {dist:.3f}, {alt:.3f}, {ang:.3f}, {weight:.3f}, {acc:.3f}, {fuel:.3f}".format(
        vs=self.vs, 
        hs=self.hs, 
        dist=self.dist, 
        alt=self.alt, 
        ang=self.ang, 
        weight=self.weight, 
        acc=self.acc,
        fuel=self.fuel)

In [0]:
class Simulator:
  def __init__(self, dt):
    self.time = 0
    self.dt = dt

  def run(self, vs, hs, dist, ang, alt):
    bereshit = Bereshit(vs, hs, dist, ang, alt)
    print('time, vert_s, horiz_s, distance, altitude, angle, weight, acceleration, fuel')
    while bereshit.alt > 0:
      info = '{time:03d}, {bereshit}'.format(time=self.time, bereshit=bereshit)
      print(info) if self.time % 10 ==0 or bereshit.alt < 100 else None
      bereshit.land()
      self.time += self.dt
    info = '{time:03d}, {bereshit}'.format(time=self.time, bereshit=bereshit)
    print(info)

In [19]:
def main():
  simulator = Simulator(1)
  simulator.run(vs=24.8, hs=932, dist=181*1000, ang=58.3, alt=13748)

main()

time, vert_s, horiz_s, distance, altitude, angle, weight, acceleration, fuel
000, 24.800, 932.000, 181000.000, 13748.000, 58.300, 286.000, 0.000, 121.000
010, 32.233, 918.085, 171749.527, 13459.339, 58.300, 284.446, 1.550, 119.446
020, 39.946, 902.543, 162654.088, 13094.823, 58.300, 282.892, 1.559, 117.892
030, 47.944, 886.919, 153714.520, 12651.613, 58.300, 281.338, 1.568, 116.338
040, 56.232, 871.211, 144931.657, 12126.832, 58.300, 279.784, 1.576, 114.784
050, 64.813, 855.420, 136306.329, 11517.563, 58.300, 278.230, 1.585, 113.230
060, 73.691, 839.547, 127839.363, 10820.852, 58.300, 276.676, 1.594, 111.676
070, 82.870, 823.592, 119531.581, 10033.705, 58.300, 275.122, 1.603, 110.122
080, 92.355, 807.555, 111383.800, 9153.087, 58.300, 273.568, 1.612, 108.568
090, 102.149, 791.437, 103396.833, 8175.924, 58.300, 272.014, 1.621, 107.014
100, 112.257, 775.239, 95571.486, 7099.098, 58.300, 270.460, 1.631, 105.460
110, 122.683, 758.961, 87908.558, 5919.450, 58.300, 268.906, 1.640, 103.906
12